In [62]:
from glob import glob
import os
import sys
from itertools import accumulate
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 14})

sys.path.append('../../')
import utils

In [63]:
def get_log(log_path):    
    exp_path = log_path.replace('log.json', 'exp.json')
    conf = utils.read_json(exp_path)
    logs = utils.read_json(log_path)

    epochs = [int(i) for i in logs['epoch'].keys()]
    new_log = {'epoch': epochs}

    for epoch in epochs:
        log = logs['epoch'][str(epoch)]

        for k, v in log.items():
            if isinstance(v, dict):
                prefix = k[:5]
                for kk, vv in v.items():
                    new_key = f'{prefix}_{kk}'
                    new_log.setdefault(new_key, []).append(vv)

            else:
                new_log.setdefault(k, []).append(v)

    runtime = [(x+y)/3600 for x, y in zip(new_log['train_run_time'], new_log['valid_run_time'])]
    time = list(accumulate(runtime))
    
    new_log['time'] = time
    new_log['max_train_dice'] = max(new_log['train_dice'])
    new_log['max_valid_dice'] = max(new_log['valid_dice'])
    new_log['max_valid_dice_index'] = new_log['valid_dice'].index(new_log['max_valid_dice'])
    new_log['name'] = conf['run_name'] + f"_vdice_{new_log['max_valid_dice']:.3f}"

    return new_log, conf


In [64]:
log_path1 = "/data/ephemeral/home/Dongjin/level2-cv-semanticsegmentation-cv-02-lv3/Baseline/Dongjin/transformers_1119/trained_models/cont/mit-b2_cont_size_1024/log.json"
log_path2 = "/data/ephemeral/home/Dongjin/level2-cv-semanticsegmentation-cv-02-lv3/Baseline/Dongjin/transformers_1120/trained_models/cont/mit-b2_cont_size_1024_cont_weight/log.json"

log_paths = [log_path1, log_path2]
df = None

for i, log_path in enumerate(log_paths):
    log, _ = get_log(log_path)
    name = log['name']
    index = log['max_valid_dice_index']
    
    dicts = {}
    dicts['name'] = [name]

    for k, v in log.items():
        if not isinstance(v, list): 
            continue
        dicts[k] = [v[index]]

    if df is None:
        df = pd.DataFrame(dicts)
    else:
        df = pd.concat([df, pd.DataFrame(dicts)], ignore_index=True)

df.to_csv('result.csv', index=False)
